# LSMS Processing

In [50]:
import pandas as pd

In [68]:
year = 2011

if year == 2015:
    LSMS_PATH = '/Users/brandon/Downloads/ETH_2015_ESS_v02_M_STATA8'
elif year == 2013:
    LSMS_PATH = '/Users/brandon/Downloads/ETH_2013_ESS_v02_M_Stata8'
elif year == 2011:
    LSMS_PATH = '/Users/brandon/Downloads/ETH_2011_ERSS_v02_M_Stata8'    

## Load Crop Yield Data

In [69]:
vars_ = pd.read_csv('Harvest_by_Field_data_dict.csv').set_index('Variable')

In [70]:
vars_dict = vars_.to_dict()

In [71]:
if year in [2013, 2015]:
    vars_to_select = ['household_id2','saq01','saq02','saq03','saq04','parcel_id','field_id','crop_name','ph_s9q04_a','ph_s9q04_b']

# 2011 did not have non-standardized (kg, g) units, so we should just use the kg variable    
elif year == 2011:
    vars_to_select = ['household_id2','saq01','saq02','saq03','saq04','parcel_id','field_id','crop_name','ph_s9q12_a']

In [72]:
if year == 2015:
    int_path = "Post-Harvest/"
    wave = 3
elif year == 2013:
    int_path = ""
    wave = 2
elif year == 2011:
    int_path = ""
    wave = 1
    
c_yields = pd.read_stata(f'{LSMS_PATH}/{int_path}sect9_ph_w{wave}.dta')

if year == 2013:
    c_yields['crop_name'] = c_yields['crop_code']
    
if year == 2011:
    c_yields['household_id2'] = c_yields['household_id']

In [73]:
c_yields = c_yields[vars_to_select]

In [74]:
c_yields = c_yields.rename(columns=vars_dict['Definition'])

In [75]:
c_yields = c_yields.set_index('Unique HH ID in wave 2')

Remove NA values from dataset (household missing)

In [76]:
c_yields = c_yields.dropna()

In [77]:
num_hh = c_yields.reset_index().groupby('Unique HH ID in wave 2')['Field ID'].count().shape[0]
num_fields = sum(c_yields.reset_index().groupby('Unique HH ID in wave 2')['Field ID'].count())

In [78]:
print(f"There are {num_hh} households with {num_fields} total fields.")

There are 1612 households with 4052 total fields.


## Load Field Area information

In [79]:
if year == 2015:
    int_path = "Post-Planting/"
    wave = 3
    path = "sect_3"
elif year == 2013:
    int_path = ""
    wave = 2
    path = "sect_3"    
elif year == 2011:
    int_path = ""
    wave = 1
    path = "sect3_"    

In [80]:
c_area = pd.read_stata(f'{LSMS_PATH}/{int_path}{path}rca_pp_w{wave}.dta')

In [81]:
if year in [2013, 2015]:
    area_vars = ['household_id2','parcel_id','field_id','pp_rcq02_a','pp_rcq02_b']

# 2011 did not have non-standardized (kg, g) units, so we should just use the kg variable    
elif year == 2011:
    area_vars = ['household_id','parcel_id','field_id','pp_rcq02_a','pp_rcq02_b']

In [82]:
c_area = c_area[area_vars].dropna(subset=['pp_rcq02_a'])

if year == 2011:
    c_area['household_id2'] = c_area['household_id']
    del(c_area['household_id'])

In [83]:
print(f"{c_area.shape[0]} total fields have area information")

5031 total fields have area information


In [84]:
new_vars = {'parcel_id': 'Parcel ID', 'field_id': 'Field ID', 'pp_rcq02_a': 'Area', 'pp_rcq02_b': 'Area (decimal)'}
c_area = c_area.rename(columns=new_vars)

In [85]:
new_df = pd.merge(c_yields.reset_index(), c_area, 
                  how='left', 
                  left_on=['Unique HH ID in wave 2','Parcel ID','Field ID'], 
                  right_on = ['household_id2','Parcel ID','Field ID'])

In [86]:
c_yields = new_df.set_index('Unique HH ID in wave 2')

## Load Geo Houshould information

In [87]:
if year == 2015:
    int_path = "Geovariables/"
    f = "ETH_HouseholdGeovars_y3.dta"
elif year == 2013:
    int_path = ""
    f = "Pub_ETH_HouseholdGeovars_Y2.dta"
elif year == 2011:
    int_path = ""
    f = "Pub_ETH_HouseholdGeovariables_Y1.dta"    
    
household_geo = pd.read_stata(f'{LSMS_PATH}/{int_path}{f}')

In [88]:
if year in [2013, 2015]:
    household_vars = {
    'household_id2':'Unique HH ID in wave 2',
    'lon_dd_mod':'EA Longitude (WGS84) Modified',
    'lat_dd_mod':'EA Latitude (WGS84) Modified'
    }
elif year == 2011:
    household_vars = {
    'household_id':'Unique HH ID in wave 2',
    'LAT_DD_MOD':'EA Latitude (WGS84) Modified',
    'LON_DD_MOD':'EA Longitude (WGS84) Modified'
    }    

In [89]:
household_geo = household_geo[household_vars.keys()].rename(columns=household_vars)

Add this geospatial data to crop yield information

In [90]:
c_yields = c_yields.join(household_geo.set_index('Unique HH ID in wave 2'), on='Unique HH ID in wave 2', how='left')

In [91]:
c_yields

,Region Code,Zone Code,Woreda Code,Kebele/FA Code,Parcel ID,Field ID,Crop Name,How much crop harvested from this field during the last season KILOS,Area,Area (decimal),household_id2,EA Latitude (WGS84) Modified,EA Longitude (WGS84) Modified
Unique HH ID in wave 2,,,,,,,,,,,,,
01010101601002,Tigray,1,1,16,1,4,SORGHUM,600.0,5946.0,89.0,01010101601002,14.353816,37.890876
01010101601002,Tigray,1,1,16,1,2,MILLET,200.0,69.0,99.0,01010101601002,14.353816,37.890876
01010101601002,Tigray,1,1,16,1,2,MILLET,200.0,7751.0,70.0,01010101601002,14.353816,37.890876
01010101601017,Tigray,1,1,16,2,1,MILLET,80.0,1525.0,16.0,01010101601017,14.353816,37.890876
01010101601017,Tigray,1,1,16,3,1,SORGHUM,600.0,6532.0,52.0,01010101601017,14.353816,37.890876
01010101601101,Tigray,1,1,16,1,5,MILLET,100.0,1594.0,47.0,01010101601101,14.353816,37.890876
01010101601101,Tigray,1,1,16,1,7,SESAME,100.0,371.0,23.0,01010101601101,14.353816,37.890876
01010101601101,Tigray,1,1,16,1,10,MILLET,200.0,4724.0,86.0,01010101601101,14.353816,37.890876
01010101601116,Tigray,1,1,16,1,2,MAIZE,8.0,115.0,12.0,01010101601116,14.353816,37.890876


In [92]:
c_yields = c_yields.reset_index()

In [93]:
c_yields.shape

(4060, 14)

### Crop Unit Conversions

In [94]:
if year == 2015:
    int_path = "Food and Crop Conversion Factors/"
    f = "Crop_CF_Wave3.dta"
elif year == 2013:
    int_path = ""
    f = "Crop_CF_Wave2.dta"
elif year == 2011:
    int_path = ""
    f = "Food_CF_Wave1.dta"    

c_conversions = pd.read_stata(f'{LSMS_PATH}/{int_path}{f}')

In [95]:
if year in [2013, 2015]:
    c_conversions_vars = {'crop_code': 'Crop Name', 'unit_cd': 'Unit Code', 'mean_cf_nat': 'Conversion Factor (National)'}
elif year == 2011:
    c_conversions_vars = {'item_cd': 'Crop Name', 'unit_cd': 'Unit Code', 'mean_cf_nat': 'Conversion Factor (National)'}
    
c_conversions = c_conversions[c_conversions_vars.keys()]
c_conversions = c_conversions.rename(columns=c_conversions_vars)

In [96]:
if year == 2015:
    c_conversions['Crop Name'] = c_conversions['Crop Name'].apply(lambda x: x.split('. ')[1])
    c_conversions['Unit Code'] = c_conversions['Unit Code'].apply(lambda x: x.split('. ')[1])

In [97]:
c_conversions = c_conversions.drop_duplicates(subset=['Crop Name','Unit Code'])

Add conversion factor to yields dataframe:

In [98]:
if year in [2013, 2015]:
    c_yields = pd.merge(c_yields,
                    c_conversions,
                    how='left',
                    left_on=['Crop Name','How much [CROP] did you harvest from this [FIELD](UNIT)'],
                    right_on=['Crop Name','Unit Code'])

Perform conversion:

In [99]:
if year in [2013, 2015]:
    c_yields['Quantity_Kilograms'] = c_yields['How much [CROP] did you harvest from this [FIELD](QUANTITY)'] * c_yields['Conversion Factor (National)']
elif year == 2011:
    c_yields['Quantity_Kilograms'] = c_yields['How much crop harvested from this field during the last season KILOS']

In [100]:
c_yields

,Unique HH ID in wave 2,Region Code,Zone Code,Woreda Code,Kebele/FA Code,Parcel ID,Field ID,Crop Name,How much crop harvested from this field during the last season KILOS,Area,Area (decimal),household_id2,EA Latitude (WGS84) Modified,EA Longitude (WGS84) Modified,Quantity_Kilograms
0,01010101601002,Tigray,1,1,16,1,4,SORGHUM,600.0,5946.0,89.0,01010101601002,14.353816,37.890876,600.0
1,01010101601002,Tigray,1,1,16,1,2,MILLET,200.0,69.0,99.0,01010101601002,14.353816,37.890876,200.0
2,01010101601002,Tigray,1,1,16,1,2,MILLET,200.0,7751.0,70.0,01010101601002,14.353816,37.890876,200.0
3,01010101601017,Tigray,1,1,16,2,1,MILLET,80.0,1525.0,16.0,01010101601017,14.353816,37.890876,80.0
4,01010101601017,Tigray,1,1,16,3,1,SORGHUM,600.0,6532.0,52.0,01010101601017,14.353816,37.890876,600.0
5,01010101601101,Tigray,1,1,16,1,5,MILLET,100.0,1594.0,47.0,01010101601101,14.353816,37.890876,100.0
6,01010101601101,Tigray,1,1,16,1,7,SESAME,100.0,371.0,23.0,01010101601101,14.353816,37.890876,100.0
7,01010101601101,Tigray,1,1,16,1,10,MILLET,200.0,4724.0,86.0,01010101601101,14.353816,37.890876,200.0
8,01010101601116,Tigray,1,1,16,1,2,MAIZE,8.0,115.0,12.0,01010101601116,14.353816,37.890876,8.0
9,01010101601116,Tigray,1,1,16,4,1,SESAME,200.0,3203.0,1.0,01010101601116,14.353816,37.890876,200.0


In [101]:
c_yields.shape

(4060, 15)

### Land Area Lookups

In [102]:
if year == 2015:
    int_path = "Land Area Conversion Factor/"
    f = "ET_local_area_unit_conversion.dta"
elif year == 2013:
    int_path = ""
    f = "ET_local_area_unit_conversion.dta"
elif year == 2011:
    int_path = ""
    f = "ET_local_area_unit_conversion.dta"    

land_conversions = pd.read_stata(f'{LSMS_PATH}/{int_path}{f}')

Remove duplicates in land conversions dataframe:

In [103]:
land_conversions = land_conversions.drop_duplicates(subset=['region','zonename','woredaname'])

In [104]:
def gen_reg_zone_woreda(row, f):
    """
    Generates a string combining region_zone_woreda for use during lookups.
    """
    if f == 'c_yields':
        region = row['Region Code']
        zone = int(row['Zone Code'])
        woreda = int(row['Woreda Code'])
        return f"{region}_{zone}_{woreda}".lower()
    elif f == 'land_conversions':
        region = row['region']
        zone = int(row['zone'])
        woreda = int(row['woreda'])
        return f"{region}_{zone}_{woreda}".lower()

In [105]:
c_yields['region_zone_woreda'] = c_yields.apply(lambda row: gen_reg_zone_woreda(row, 'c_yields'), axis=1)

In [106]:
land_conversions['region_zone_woreda'] = land_conversions.apply(lambda row: gen_reg_zone_woreda(row, 'land_conversions'), axis=1)

In [107]:
c_yields.shape

(4060, 16)

In [108]:
c_yields = c_yields.merge(land_conversions, on='region_zone_woreda', how='left')

In [109]:
c_yields.shape

(4060, 23)

In [110]:
c_yields['Area_Normalized'] = c_yields['Area'] * c_yields['conversion']

### Subset Crop Yields

In [111]:
c_yields_out = c_yields[['Unique HH ID in wave 2','region_zone_woreda','Crop Name','Quantity_Kilograms','EA Longitude (WGS84) Modified','EA Latitude (WGS84) Modified','Area_Normalized']]

In [112]:
out_vars = {'Unique HH ID in wave 2': 'Household_UUID',
            'zonename': 'Zone',
            'woredaname': 'Woreda',
            'Crop Name': 'Crop Name',
            'Quantity_Kilograms': 'Quantity (kg)',
            'EA Longitude (WGS84) Modified': 'Lon',
            'EA Latitude (WGS84) Modified': 'Lat',
            'Area_Normalized': 'Area (Normalized)'}

In [113]:
c_yields_out = c_yields_out.rename(columns=out_vars)
c_yields_out['Crop Name'] = c_yields_out['Crop Name'].apply(lambda x: str(x).title())

In [114]:
c_yields_out['Year'] = year

In [115]:
c_yields_out.to_csv(f'LSMS_{year}.csv', index=False)